# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("/home/scott/Documents/ML_WASH/regression/week2/kc_house_data.csv")
train = pd.read_csv("/home/scott/Documents/ML_WASH/regression/week2/kc_house_train_data.csv")
test = pd.read_csv("/home/scott/Documents/ML_WASH/regression/week2/kc_house_test_data.csv")

In [4]:
np.log(25)

3.2188758248682006

In [5]:
for df in [train, test]:
    df['bedrooms_squared'] = df.bedrooms * df.bedrooms # increases importance of bedrooms, punishes few bedrooms
    df['bed_bath_rooms'] = df.bedrooms * df.bathrooms # large when both large, 'interaction' variable
    df['log_sqft_living'] = df.sqft_living.apply(lambda x: np.log(x)) # log spreads out small numbers and clusters big together
    # log rises quickly for small numbers, and then rises very slowly
    df['lat_plus_long'] = df.lat + df.long

In [66]:
for col in ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']:
    print(round( np.mean(test[col]), 2))

12.45
7.5
7.55
-74.65


In [76]:
feat1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
feat2 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
feat3 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [77]:
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression()
model1 = reg1.fit(train[feat1], train.price)

reg2 = LinearRegression()
model2 = reg2.fit(train[feat2], train.price)

reg3 = LinearRegression()
model3 = reg3.fit(train[feat3], train.price)

In [78]:
model1.intercept_, model2.intercept_, model3.intercept_

(-69075726.79256986, -66867968.871078975, -62036084.98609827)

In [85]:
print(model1.coef_, '\n\n', model2.coef_, '\n\n', model3.coef_)

[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05] 

 [ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04] 

 [ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
  1.27334900e+05]


In [88]:
from sklearn.metrics import mean_squared_error

rss1 = mean_squared_error(train.price, model1.predict(train[feat1]))
rss2 = mean_squared_error(train.price, model2.predict(train[feat2]))  
rss3 = mean_squared_error(train.price, model3.predict(train[feat3]))  

In [90]:
pd.DataFrame( [rss1, rss2, rss3])

,0
0,5.567648e+10
1,5.513228e+10
2,5.196942e+10


In [91]:
rss1 = mean_squared_error(test.price, model1.predict(test[feat1]))
rss2 = mean_squared_error(test.price, model2.predict(test[feat2]))  
rss3 = mean_squared_error(test.price, model3.predict(test[feat3]))  

In [92]:
pd.DataFrame( [rss1, rss2, rss3])

,0
0,5.332241e+10
1,5.282040e+10
2,6.129967e+10


# Fire up Turi Create

In [ ]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [ ]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [ ]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [ ]:
example_weight_summary = example_model.coefficients
print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.